In [1]:
import pandas as pd

In [17]:
# data source: 
# TODO: function to read csv
# put csv in "__init__ main" to execute
china = pd.read_csv('FluNetInteractiveReport_China.csv', header = 2)
usa = pd.read_csv('FluNetInteractiveReport_USA.csv', header = 2)
uk = pd.read_csv('FluNetInteractiveReport_UK.csv', header = 2)
#uk
#china
#usa

,Country,WHOREGION,FLUREGION,Year,Week,SDATE,EDATE,SPEC_RECEIVED_NB,SPEC_PROCESSED_NB,AH1,...,AH5,ANOTSUBTYPED,INF_A,BYAMAGATA,BVICTORIA,BNOTDETERMINED,INF_B,ALL_INF,ALL_INF2,TITLE
0,United Kingdom of Great Britain and Northern I...,European Region of WHO,Northern Europe,2019,1,2018-12-31,2019-01-06,NaN,2550.0,0.0,...,NaN,1451.0,1965.0,0.0,0.0,22.0,22.0,1987.0,NaN,No Report
1,United Kingdom of Great Britain and Northern I...,European Region of WHO,Northern Europe,2019,2,2019-01-07,2019-01-13,NaN,2960.0,0.0,...,NaN,2067.0,2740.0,0.0,0.0,42.0,42.0,2782.0,NaN,No Report
2,United Kingdom of Great Britain and Northern I...,European Region of WHO,Northern Europe,2019,3,2019-01-14,2019-01-20,NaN,2839.0,0.0,...,NaN,1979.0,2624.0,0.0,0.0,31.0,31.0,2655.0,NaN,No Report
3,United Kingdom of Great Britain and Northern I...,European Region of WHO,Northern Europe,2019,4,2019-01-21,2019-01-27,NaN,2704.0,0.0,...,NaN,2378.0,3030.0,0.0,0.0,29.0,29.0,3059.0,NaN,No Report
4,United Kingdom of Great Britain and Northern I...,European Region of WHO,Northern Europe,2019,5,2019-01-28,2019-02-03,NaN,2896.0,0.0,...,NaN,2687.0,3580.0,0.0,0.0,43.0,43.0,3623.0,NaN,No Report
5,United Kingdom of Great Britain and Northern I...,European Region of WHO,Northern Europe,2019,6,2019-02-04,2019-02-10,NaN,3064.0,0.0,...,NaN,2961.0,4015.0,0.0,0.0,19.0,19.0,4034.0,NaN,No Report
6,United Kingdom of Great Britain and Northern I...,European Region of WHO,Northern Europe,2019,7,2019-02-11,2019-02-17,NaN,2921.0,0.0,...,NaN,3061.0,3885.0,0.0,0.0,38.0,38.0,3923.0,NaN,No Report
7,United Kingdom of Great Britain and Northern I...,European Region of WHO,Northern Europe,2019,8,2019-02-18,2019-02-24,NaN,2587.0,0.0,...,NaN,2167.0,2747.0,0.0,0.0,24.0,24.0,2771.0,NaN,No Report
8,United Kingdom of Great Britain and Northern I...,European Region of WHO,Northern Europe,2019,9,2019-02-25,2019-03-03,NaN,1975.0,0.0,...,NaN,1390.0,1698.0,0.0,0.0,21.0,21.0,1719.0,NaN,No Report
9,United Kingdom of Great Britain and Northern I...,European Region of WHO,Northern Europe,2019,10,2019-03-04,2019-03-10,NaN,1889.0,0.0,...,NaN,940.0,1190.0,0.0,0.0,15.0,15.0,1205.0,NaN,No Report


In [20]:
# data source: https://trends.google.com/trends/explore?date=2019-01-01%202020-10-27&geo=US&q=mask
# TODO: function to analyze mask google trend
# read and clean

In [29]:
# data source: https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide
# TODO: function to analyze covid case
# read and clean

#covid = pd.read_csv('COVID-19-worldwide.xlsx', sep = '\t')
#covid  ((這邊還沒讀進去))

# 變成dataframe然後取出個國家的確認數&日期
# 看要不要group by年、月

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa5 in position 1: invalid start byte

In [22]:
# columns we want: year, month, ALL_INF, mask google trend
#TODO: function to make a separte column of datetime
# and then group by 1. year and 2. month (2 conditions)   ('ALL_INF')

In [25]:
# aggregrate everything by country
# columns: year, month, mask, flu, covid

In [26]:
# Data visualization
# import matplotlib as plt